In [ ]:
#Importer les modules:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
import colorama
from colorama import Fore
from colorama import Style
import os, sys
import matplotlib
from matplotlib import cm
from matplotlib.patches import Circle, Wedge, Rectangle
from math import log10

import scipy.stats as stats

import statsmodels.api as sm
from statsmodels.formula.api import ols


import ast
import scipy.stats as stats
import re
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
from sklearn.inspection import permutation_importance
import math
from datetime import datetime

In [2]:
print(pd.__version__)

1.3.4


In [ ]:
#A. IMPORTER LES DONNÉES:

In [3]:
cd Desktop\Data Science candidature\Projets\Projet 4\Data\

C:\Users\'titite\Desktop\Data Science candidature\Projets\Projet 4\Data


In [4]:
ls

 Le volume dans le lecteur C s'appelle OS
 Le num‚ro de s‚rie du volume est 609B-8774

 R‚pertoire de C:\Users\'titite\Desktop\Data Science candidature\Projets\Projet 4\Data

12/06/2022  11:32    <DIR>          .
12/06/2022  11:32    <DIR>          ..
01/07/2021  00:01         1ÿ588ÿ303 2015-building-energy-benchmarking.csv
01/07/2021  00:01         1ÿ235ÿ434 2016-building-energy-benchmarking.csv
26/03/2022  18:13           770ÿ246 archive.zip
14/06/2022  00:22           391ÿ478 Data_clean.csv
14/06/2022  00:22           466ÿ450 Data_clean_bins.csv
14/06/2022  00:22           271ÿ956 Data_clean_E.csv
01/07/2021  00:01            54ÿ630 socrata_metadata_2015-building-energy-benchmarking.json
01/07/2021  00:01            45ÿ816 socrata_metadata_2016-building-energy-benchmarking.json
               8 fichier(s)        4ÿ824ÿ313 octets
               2 R‚p(s)  152ÿ532ÿ602ÿ880 octets libres


In [5]:
df=pd.read_csv('Data_clean.csv', sep=',', low_memory=False)

In [6]:
df.count()

OSEBuildingID                    3247
TaxParcelIdentificationNumber    3246
PropertyName                     3247
SourceEUI_kBtu_sf_log            3247
TotalGHGEmissions_log            3247
BuildingType                     3247
YearBuilt                        3247
PropertyGFATotal                 3247
Neighborhood                     3247
ZipCode_New                      3247
dtype: int64

In [7]:
df.dtypes

OSEBuildingID                      int64
TaxParcelIdentificationNumber     object
PropertyName                      object
SourceEUI_kBtu_sf_log            float64
TotalGHGEmissions_log            float64
BuildingType                      object
YearBuilt                          int64
PropertyGFATotal                   int64
Neighborhood                      object
ZipCode_New                        int64
dtype: object

In [8]:
df.head(5)

,OSEBuildingID,TaxParcelIdentificationNumber,PropertyName,SourceEUI_kBtu_sf_log,TotalGHGEmissions_log,BuildingType,YearBuilt,PropertyGFATotal,Neighborhood,ZipCode_New
0,1,659000030,MAYFLOWER PARK HOTEL,5.161925,5.523179,NonResidential,1927,88434,DOWNTOWN,98101
1,2,659000220,PARAMOUNT HOTEL,5.259057,5.577879,NonResidential,1996,103566,DOWNTOWN,98101
2,3,659000475,WESTIN HOTEL,5.495938,7.631664,NonResidential,1969,961990,DOWNTOWN,98101
3,8,659000970,WARWICK SEATTLE HOTEL,5.437209,6.231858,NonResidential,1980,119890,DOWNTOWN,98121
4,9,660000560,WEST PRECINCT (SEATTLE POLICE),5.750984,5.722343,Nonresidential COS,1999,97288,DOWNTOWN,98101


In [ ]:
#On a, parmi les variables descriptives 2 variables catégoriques: 'BuildingType' et 'Neighborhood', il faut les rendre
#continues en utilisant OneHotEncoder.

#B. OneHotEncoder:

#B.1. On commence par lancer le OneHotEncoder dans les variables catégorielles.

In [9]:
encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)

In [ ]:
#B.1.1 'BuildingType':

In [10]:
encoder_BuildingType = pd.DataFrame(encoder.fit_transform(df[['BuildingType']]))
encoder_BuildingType

,0,1,2,3,4
0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...
3242,0.0,1.0,0.0,0.0,0.0
3243,0.0,1.0,0.0,0.0,0.0
3244,0.0,1.0,0.0,0.0,0.0
3245,0.0,1.0,0.0,0.0,0.0


In [ ]:
#On renomme les variables catégorielles pour pouvoir mieux se retrouver.
#Notez que OneHotEncoder prend en considération les différentes valeurs d'une colonne par ordre alphabétique.

In [11]:
sorted(df['BuildingType'].unique())

['Campus',
 'NonResidential',
 'Nonresidential COS',
 'Nonresidential WA',
 'SPS-District K-12']

In [12]:
encoder_BuildingType.rename(columns={0:'Campus',1:'Multifamily_HR',2:'Multifamily_LR',3:'Multifamily_MR', 4:'NonResidential', 5:'NonResidential_COS', 6:'NonResidential_WA', 7:'SPS-District_K-12'},inplace = True)
encoder_BuildingType

,Campus,Multifamily_HR,Multifamily_LR,Multifamily_MR,NonResidential
0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...
3242,0.0,1.0,0.0,0.0,0.0
3243,0.0,1.0,0.0,0.0,0.0
3244,0.0,1.0,0.0,0.0,0.0
3245,0.0,1.0,0.0,0.0,0.0


In [ ]:
#Vérification.

In [13]:
encoder_BuildingType.sum()

Campus              46.0
Multifamily_HR    2874.0
Multifamily_LR     136.0
Multifamily_MR       1.0
NonResidential     190.0
dtype: float64

In [ ]:
#On retrouve bien le nombre de bâtiments par catégorie qu'on avait lors de l'analyse exploratoire.

#B.1.2 'Neighborhood':

In [14]:
encoder_Neighborhood = pd.DataFrame(encoder.fit_transform(df[['Neighborhood']]))
encoder_Neighborhood

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3242,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3244,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3245,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#On renomme les variables catégorielles pour pouvoir mieux se retrouver.
#Notez que OneHotEncoder prend en considération les différentes valeurs d'une colonne par ordre alphabétique.

In [15]:
sorted(df['Neighborhood'].unique())

['BALLARD',
 'CENTRAL',
 'DELRIDGE',
 'DOWNTOWN',
 'EAST',
 'GREATER DUWAMISH',
 'LAKE UNION',
 'MAGNOLIA / QUEEN ANNE',
 'NORTH',
 'NORTHEAST',
 'NORTHWEST',
 'SOUTHEAST',
 'SOUTHWEST']

In [16]:
encoder_Neighborhood.rename(columns={0:'BALLARD',1:'CENTRAL',2:'DELRIDGE',3:'DOWNTOWN', 4:'EAST', 5:'GREATER_DUWAMISH', 6:'LAKE_UNION', 7:'MAGNOLIA_QUEEN_ANNE', 8:'NORTH', 9:'NORTHEAST', 10:'NORTHWEST', 11:'SOUTHEAST', 12:'SOUTHWEST' },inplace = True)
encoder_Neighborhood

,BALLARD,CENTRAL,DELRIDGE,DOWNTOWN,EAST,GREATER_DUWAMISH,LAKE_UNION,MAGNOLIA_QUEEN_ANNE,NORTH,NORTHEAST,NORTHWEST,SOUTHEAST,SOUTHWEST
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3242,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3244,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3245,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Vérification.

In [17]:
encoder_Neighborhood.sum()

BALLARD                135.0
CENTRAL                103.0
DELRIDGE                91.0
DOWNTOWN               711.0
EAST                   240.0
GREATER_DUWAMISH       676.0
LAKE_UNION             290.0
MAGNOLIA_QUEEN_ANNE    292.0
NORTH                  129.0
NORTHEAST              236.0
NORTHWEST              173.0
SOUTHEAST               91.0
SOUTHWEST               80.0
dtype: float64

In [ ]:
#On retrouve bien le nombre de bâtiments par catégorie qu'on avait lors de l'analyse exploratoire.
#--------------------------------------------------------------------------------------------------------------------------

#B.2 On fusionne les colonnes résultant du OneHotEncoder dans le tableau initial.

In [18]:
f1_df = df.join(encoder_BuildingType)

In [19]:
f2_df=df.join(encoder_Neighborhood)

In [20]:
final_df= f1_df.join(encoder_Neighborhood)
final_df.count()

OSEBuildingID                    3247
TaxParcelIdentificationNumber    3246
PropertyName                     3247
SourceEUI_kBtu_sf_log            3247
TotalGHGEmissions_log            3247
BuildingType                     3247
YearBuilt                        3247
PropertyGFATotal                 3247
Neighborhood                     3247
ZipCode_New                      3247
Campus                           3247
Multifamily_HR                   3247
Multifamily_LR                   3247
Multifamily_MR                   3247
NonResidential                   3247
BALLARD                          3247
CENTRAL                          3247
DELRIDGE                         3247
DOWNTOWN                         3247
EAST                             3247
GREATER_DUWAMISH                 3247
LAKE_UNION                       3247
MAGNOLIA_QUEEN_ANNE              3247
NORTH                            3247
NORTHEAST                        3247
NORTHWEST                        3247
SOUTHEAST   

In [ ]:
#--------------------------------------------------------------------------------------------------------------------------

#C.Le dataset.

#C.1 Élimination des variables non nécessaires.

#On élimine les colonnes des variables catégorielles initiales et celles d'identification des bâtiments:
#'OSEBuildingID','TaxParcelIdentificationNumber','PropertyName','BuildingType','Neighborhood'.
#On reste aussi uniquement avec la variable cible qui nous intéresse: 'SourceEUI_kBtu_sf_log'.

In [21]:
final_df.drop({'TotalGHGEmissions_log','OSEBuildingID','TaxParcelIdentificationNumber','PropertyName','BuildingType','Neighborhood'}, axis=1, inplace=True)

In [ ]:
#f1_df.drop({'TotalGHGEmissions_log','OSEBuildingID','TaxParcelIdentificationNumber','PropertyName','BuildingType','Neighborhood'}, axis=1, inplace=True)

In [ ]:
#f2_df.drop({'TotalGHGEmissions_log','OSEBuildingID','TaxParcelIdentificationNumber','PropertyName','BuildingType','Neighborhood'}, axis=1, inplace=True)

In [ ]:
#C.2 Le dataset.

In [22]:
dataset=final_df
dataset.head(5)

,SourceEUI_kBtu_sf_log,YearBuilt,PropertyGFATotal,ZipCode_New,Campus,Multifamily_HR,Multifamily_LR,Multifamily_MR,NonResidential,BALLARD,...,DOWNTOWN,EAST,GREATER_DUWAMISH,LAKE_UNION,MAGNOLIA_QUEEN_ANNE,NORTH,NORTHEAST,NORTHWEST,SOUTHEAST,SOUTHWEST
0,5.161925,1927,88434,98101,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.259057,1996,103566,98101,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.495938,1969,961990,98101,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.437209,1980,119890,98121,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.750984,1999,97288,98101,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#C.3 Training et Test sets.

#On divise les données en 2 sets: un "training set" et un "test set".

In [23]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
train_dataset

,SourceEUI_kBtu_sf_log,YearBuilt,PropertyGFATotal,ZipCode_New,Campus,Multifamily_HR,Multifamily_LR,Multifamily_MR,NonResidential,BALLARD,...,DOWNTOWN,EAST,GREATER_DUWAMISH,LAKE_UNION,MAGNOLIA_QUEEN_ANNE,NORTH,NORTHEAST,NORTHWEST,SOUTHEAST,SOUTHWEST
1559,4.144721,1904,72859,98108,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
716,5.249127,1960,20539,98107,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1726,3.242592,2009,89821,98118,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2622,3.490428,1960,36498,98119,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
298,6.816626,2001,378443,98109,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,5.139322,2007,64015,98108,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
288,5.757007,1985,193500,98121,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2699,4.565389,1952,41308,98134,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1181,5.902907,1920,31332,98116,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [24]:
train_dataset.sum()

SourceEUI_kBtu_sf_log    1.250961e+04
YearBuilt                5.096048e+06
PropertyGFATotal         3.004196e+08
ZipCode_New              2.549069e+08
Campus                   3.500000e+01
Multifamily_HR           2.303000e+03
Multifamily_LR           1.070000e+02
Multifamily_MR           1.000000e+00
NonResidential           1.520000e+02
BALLARD                  1.080000e+02
CENTRAL                  8.800000e+01
DELRIDGE                 7.100000e+01
DOWNTOWN                 5.870000e+02
EAST                     1.900000e+02
GREATER_DUWAMISH         5.210000e+02
LAKE_UNION               2.340000e+02
MAGNOLIA_QUEEN_ANNE      2.360000e+02
NORTH                    9.500000e+01
NORTHEAST                1.850000e+02
NORTHWEST                1.400000e+02
SOUTHEAST                7.700000e+01
SOUTHWEST                6.600000e+01
dtype: float64

In [ ]:
#C.4 Quelques statistiques.

In [25]:
train_stats = train_dataset.describe()
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
SourceEUI_kBtu_sf_log,2598.0,4.815094,0.894457,0.0,4.360548,4.891476,5.31959,7.828874e+00
YearBuilt,2598.0,1961.527329,32.799983,1900.0,1929.250000,1965.000000,1989.00000,2.015000e+03
PropertyGFATotal,2598.0,115634.953811,262326.888549,16300.0,30152.000000,50017.000000,104313.25000,9.320156e+06
ZipCode_New,2598.0,98116.581601,17.223689,98006.0,98104.000000,98109.000000,98122.00000,9.819900e+04
Campus,2598.0,0.013472,0.115306,0.0,0.000000,0.000000,0.00000,1.000000e+00
Multifamily_HR,2598.0,0.886451,0.317324,0.0,1.000000,1.000000,1.00000,1.000000e+00
Multifamily_LR,2598.0,0.041186,0.198757,0.0,0.000000,0.000000,0.00000,1.000000e+00
Multifamily_MR,2598.0,0.000385,0.019619,0.0,0.000000,0.000000,0.00000,1.000000e+00
NonResidential,2598.0,0.058507,0.234744,0.0,0.000000,0.000000,0.00000,1.000000e+00
BALLARD,2598.0,0.041570,0.199644,0.0,0.000000,0.000000,0.00000,1.000000e+00


In [ ]:
#C.5 Normalisation des données.

#Il est recommandé de normaliser les variables qui ont différentes échelles (cf. statistiques en haut). En fait, même si 
#le modèle peut converger sans normalisation ça rend l'entraînement plus difficile et le résultat dépendant des unités 
#utilisées pour les variables.

#Notez que nous avons générer les statistiques précédentes uniquement sur le "set training", ces statistiques sont également
#utilisées pour normaliser l'ensemble de données de test, ceci l'ensemble de données de test doivent être projetées dans la 
#même distribution que celle sur laquelle le training fut fait.

#On a décidé de normaliser aussi la variable cible. En fait même si ceci n'est pas vraiment nécessaire, c'est juste que des 
#fois si on ne normalise pas la variable cible il peut y avoir des bias et on a préféré d'éviter ceci depuis le début.

In [26]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
train_dataset_n = norm(train_dataset)
test_dataset_n = norm(test_dataset)

In [27]:
train_stats_n = train_dataset_n.describe()
train_stats_n = train_stats_n.transpose()
train_stats_n

,count,mean,std,min,25%,50%,75%,max
SourceEUI_kBtu_sf_log,2598.0,8.847191e-15,1.0,-5.383257,-0.508181,0.085396,0.564025,3.369396
YearBuilt,2598.0,-5.314584e-16,1.0,-1.875834,-0.984065,0.105874,0.837582,1.630265
PropertyGFATotal,2598.0,1.210968e-17,1.0,-0.378669,-0.325864,-0.250138,-0.043159,35.087982
ZipCode_New,2598.0,-3.505173e-13,1.0,-6.420321,-0.730482,-0.440185,0.314590,4.785177
Campus,2598.0,4.441640e-16,1.0,-0.116836,-0.116836,-0.116836,-0.116836,8.555722
Multifamily_HR,2598.0,-8.931355e-18,1.0,-2.793524,0.357833,0.357833,0.357833,0.357833
Multifamily_LR,2598.0,3.694333e-17,1.0,-0.207215,-0.207215,-0.207215,-0.207215,4.824045
Multifamily_MR,2598.0,4.746679e-16,1.0,-0.019619,-0.019619,-0.019619,-0.019619,50.950961
NonResidential,2598.0,1.198896e-16,1.0,-0.249236,-0.249236,-0.249236,-0.249236,4.010725
BALLARD,2598.0,3.077044e-16,1.0,-0.208223,-0.208223,-0.208223,-0.208223,4.800696


In [ ]:
#C.6 Séparez la variable cible des variables prédictives. 

#La variable cible est la valeur que l'on entraînera le modèle à prédire.

In [28]:
train_cible_n = train_dataset_n.pop('SourceEUI_kBtu_sf_log')
test_cible_n = test_dataset_n.pop('SourceEUI_kBtu_sf_log')

In [ ]:
#--------------------------------------------------------------------------------------------------------------------------

#D. Les Modèles.
#Construction et entraînement des modèles.
    #Notez que la fonction ".fit", c'est celle qui est chargée d'entraîner notre modèle et que la fonction ".predict", c'est
    #celle qui est chargée de prédire la variable cible à partir du dataset test.
    
    
#------

#D.1 DummyRegressor: 
    #On prend la médiane car elle est statistiquement préférable à la moyenne.
    #Sans optimisation des hyperparamètres (pour le DummyRegressor il n'y a pas vraiment des paramètres à optimiser).

In [29]:
DM= DummyRegressor(strategy='median')

In [ ]:
#D.1.1 Entraînement du modèle.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [30]:
start_time = datetime.now()

In [31]:
DM.fit(train_dataset_n,train_cible_n)

DummyRegressor(strategy='median')

In [32]:
end_time = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [33]:
DM_time = end_time - start_time

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [34]:
DM.score(train_dataset_n,train_cible_n)

-0.007295213248882604

In [ ]:
#Notez que la valeur absolue du R^2 (qui est donné par le "score") est loin de 1, ce qui montre la faiblesse de ce 
#modèle.

#D.1.2 Prédiction.

In [35]:
DM_prediction=DM.predict(test_dataset_n)
DM_prediction

array([0.08539558, 0.08539558, 0.08539558, 0.08539558, 0.08539558,
       0.08539558, 0.08539558, 0.08539558, 0.08539558, 0.08539558,
       0.08539558, 0.08539558, 0.08539558, 0.08539558, 0.08539558,
       0.08539558, 0.08539558, 0.08539558, 0.08539558, 0.08539558,
       0.08539558, 0.08539558, 0.08539558, 0.08539558, 0.08539558,
       0.08539558, 0.08539558, 0.08539558, 0.08539558, 0.08539558,
       0.08539558, 0.08539558, 0.08539558, 0.08539558, 0.08539558,
       0.08539558, 0.08539558, 0.08539558, 0.08539558, 0.08539558,
       0.08539558, 0.08539558, 0.08539558, 0.08539558, 0.08539558,
       0.08539558, 0.08539558, 0.08539558, 0.08539558, 0.08539558,
       0.08539558, 0.08539558, 0.08539558, 0.08539558, 0.08539558,
       0.08539558, 0.08539558, 0.08539558, 0.08539558, 0.08539558,
       0.08539558, 0.08539558, 0.08539558, 0.08539558, 0.08539558,
       0.08539558, 0.08539558, 0.08539558, 0.08539558, 0.08539558,
       0.08539558, 0.08539558, 0.08539558, 0.08539558, 0.08539

In [ ]:
#D.1.3 Calcul de différentes métriques suite à la prédiction du modèle.

#Calcul du R^2:

In [36]:
R2_DM=r2_score(test_cible_n,DM_prediction)
R2_DM

-0.004197135237299365

In [ ]:
#Calcul du MSE:

In [37]:
MSE_DM=mean_squared_error(test_cible_n,DM_prediction)

In [ ]:
#Calcul du RMSE:

In [38]:
RMSE_DM=math.sqrt(MSE_DM)

In [ ]:
#Calcul du MAE:

In [39]:
 MAE_DM=mean_absolute_error(test_cible_n,DM_prediction)

In [ ]:
#D.1.5 Résultats.

In [40]:
results_DM= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_DM= results_DM.append({'Modèle':'Dummy Regressor','R^2':R2_DM,'MSE':MSE_DM,'RMSE':RMSE_DM,'MAE':MAE_DM ,'Temps de calcul (s)':DM_time.seconds},ignore_index=True)
results_DM

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Dummy Regressor,-0.004197,1.035714,1.0177,0.735766,1


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1 (des valeurs de R^2 proches de 1 montrent quele modèle peut prédire relativement 
#  les données avec précision)
# -la valeur du RMSE est supérieure à 0.5 (des valeurs de RMSE comprises entre 0.2 et 0.5 montrent que le modèle peut 
#  prédire relativement les données avec précision) et
# -La valeur du MAE de modèle naïf de Random forest servira de base, on rappelle qu'il n'y a pas de valeur correcte pour la 
#  MSE. En termes simples, plus la valeur est faible, mieux c'est et 0 signifie que le modèle est parfait. On se base dans
#  la MEA d'un modèle naïf (comme le dummy regressor) pour comparer avec la MEA de modèles plus performants.

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle.


#-----------------------------------------------------------------------

#D.2 LinearRegression.
    #Sans optimisation des hyperparamètres (pour la régression linéaire il n'y a pas vraiment des paramètres à optimiser).

In [41]:
LR=LinearRegression()

In [42]:
LR.get_params()

{'copy_X': True,
 'fit_intercept': True,
 'n_jobs': None,
 'normalize': False,
 'positive': False}

In [ ]:
#D.2.1 Entraînement du modèle.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [43]:
start_time = datetime.now()

In [44]:
LR.fit(train_dataset_n,train_cible_n)

LinearRegression()

In [45]:
end_time = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [46]:
LR_time = end_time - start_time

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [47]:
LR.score(train_dataset_n,train_cible_n)

0.14797826450507712

In [ ]:
#D.2.2 Prédiction.

In [48]:
LR_prediction=LR.predict(test_dataset_n)
LR_prediction

array([ 1.04930432e-01,  3.96667209e-01, -4.93237789e-02,  4.30110646e-01,
        4.74110483e-01,  6.02973069e-01, -1.02068243e+00,  5.57204270e-01,
        6.03193881e-01,  1.62105336e-01, -9.04590000e-01, -6.61639958e-02,
       -4.06274871e-01,  8.94241611e-01, -1.36958699e-01, -8.30174002e-01,
       -1.03659097e-01, -5.54945333e-01, -6.48437486e-01, -2.52376002e-01,
       -6.78229318e-01, -7.90575501e-02, -2.90385196e-01,  2.57592700e-03,
        6.63923535e-01,  4.72624457e-01,  6.21674906e-01, -4.43762870e-01,
       -3.15531950e-02, -6.48138847e-01, -8.87913174e-01,  3.04168448e-01,
        4.97744694e-01,  9.58746573e-02,  3.06408555e-02,  6.35213065e-01,
       -8.30682961e-02,  3.83138463e-01,  6.85952770e-01,  4.31304682e-01,
       -1.63722922e-01,  5.30007141e-01, -3.38244073e-02,  1.22931632e-01,
        1.24263709e-01,  5.53788336e-01,  1.09062116e-01,  5.67533303e-01,
        5.79160190e-01,  6.63040428e-01,  4.55092763e-01,  1.93418305e-01,
        5.23117230e-01, -

In [ ]:
#D.2.3 Calcul de différentes métriques suite à la prédiction du modèle.

#Calcul du R^2:

In [49]:
R2_LR=r2_score(test_cible_n,LR_prediction)
R2_LR

0.1449158351519697

In [ ]:
#Calcul du MSE:

In [50]:
MSE_LR=mean_squared_error(test_cible_n,LR_prediction)

In [ ]:
#Calcul du RMSE:

In [51]:
RMSE_LR=math.sqrt(MSE_LR)

In [ ]:
#Calcul du MAE:

In [52]:
 MAE_LR=mean_absolute_error(test_cible_n,LR_prediction)

In [ ]:
#D.2.5 Résultats.

In [53]:
results_LR= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_LR= results_LR.append({'Modèle':'Linear regression','R^2':R2_LR,'MSE':MSE_LR,'RMSE':RMSE_LR,'MAE':MAE_LR ,'Temps de calcul (s)':LR_time.seconds},ignore_index=True)
results_LR

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Linear regression,0.144916,0.881921,0.939107,0.670013,1


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1, mais plus proche aue celle du dummy regressor.
# -la valeur du RMSE est supérieure à 0.5, mais moins que le dummy regressor.
# -La valeur du MAE est meilleure que celle du modèle naïf de dummy regressor(ie. plus petite).

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle, qui est quand même meilleur que le 
#  dummy regressor.


#D.2.6 Coefficients obtenus par le modèle:

In [54]:
pd.DataFrame(LR.coef_, train_dataset_n.columns, columns = ['Coeff'])

,Coeff
YearBuilt,0.217155
PropertyGFATotal,0.025527
ZipCode_New,-0.053579
Campus,0.066117
Multifamily_HR,0.046679
Multifamily_LR,0.088907
Multifamily_MR,0.008726
NonResidential,-0.171584
BALLARD,-0.032914
CENTRAL,-0.026442


In [ ]:
#-----------------------------------------------------------------------

#D.3 ElasticNet (méthode de régression linéaire).

    #C'est une extension de la régression linéaire, avec un paramètre de pénalité supplémentaire qui vise à minimiser la 
    #complexité et/ou à réduire le nombre de variables (features) utilisées dans le modèle final, ce qui rend intéressant 
    #le fait de le tester. Quoi qu'il en soit, 'objectif -comme pour de nombreux modèles- est de réduire l'erreur de 
    #l'ensemble de test.
    #ElasticNet est un mélange de Lasso (qui inclu la pénalité L1) et Ridge Regression (qui inclue la pénalité L2).

In [55]:
EN=ElasticNet()

In [56]:
EN.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'l1_ratio': 0.5,
 'max_iter': 1000,
 'normalize': False,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

In [ ]:
#Le modèle ElasticNet est:
#(1 / (2 * n_samples) )* ||y - Xw||^2_2 + alpha * l1_ratio * ||w||_1 + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2

#Les hyperparamètres intéressant à être modifiés sont:

# -alpha:= attribue le poids accordé à chacune des pénalités L1 et L2.

# -l1_ratio:= C'est un paramètre de pénalité de mélange. Pour l1_ratio = 0 la pénalité est une pénalité L2 (Ridge). 
#             Pour l1_ratio = 1 c'est une pénalité L1 (Lasso). Pour 0 < l1_ratio < 1, la pénalité est une combinaison de L1 
#             et L2.

# -tol:= La tolérance d'optimisation, ie. quand le modèle arrête-t-il le processus d'optimisation.



#D.3.0 Optimisation des hyperparamètres.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [57]:
start_time = datetime.now()

In [58]:
parameters = {'tol' : [0.1,0.01,0.001,0.0001], 'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100], 'l1_ratio': np.arange(0.0, 1.0, 0.1)} #0: Ridge; 1:Lasso

EN = GridSearchCV(estimator = ElasticNet(), param_grid = parameters, cv=5, verbose=False)

In [ ]:
#Notez que dans le GridSearchCV 'cv' c'est le nombre de plis (ie. d'itérations).

#D.3.1 Entraînement du modèle.

In [59]:
EN.fit(train_dataset_n,train_cible_n)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 855.7041175597905, tolerance: 202.9894269562829
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 901.2205775044621, tolerance: 211.14661459609243
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 901.6927293462292, tolerance: 209.26648727638255
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarn

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.48627172846022404, tolerance: 0.20926648727638253
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.4531999456212361, tolerance: 0.21292811112515267
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.3036022015735398, tolerance: 0.21114661459609244
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Converg

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 895.6425530764919, tolerance: 2.1292811112515264
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 855.8308298817884, tolerance: 0.2029894269562829
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 901.3501048695839, tolerance: 0.21114661459609244
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 868.5154249985769, tolerance: 20.298942695628288
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 914.2648597777144, tolerance: 21.11466145960924
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 913.548732287517, tolerance: 20.926648727638252
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarni

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 973.1639127070376, tolerance: 0.21114661459609244
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 967.7693709210695, tolerance: 0.20926648727638253
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 932.3076924632314, tolerance: 0.20239291072769433
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Convergence

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1044.4984974454387, tolerance: 20.926648727638252
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1010.185563877045, tolerance: 20.239291072769433
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1062.5063891519817, tolerance: 21.292811112515263
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceW

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                         'l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                         'tol': [0.1, 0.01, 0.001, 0.0001]},
             verbose=False)

In [60]:
end_time = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [61]:
EN_time = end_time - start_time

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [62]:
EN.score(train_dataset_n,train_cible_n)

0.1472990222486017

In [ ]:
#Les meilleurs hyperparamètres.

In [63]:
EN.best_params_

{'alpha': 0.01, 'l1_ratio': 0.6000000000000001, 'tol': 0.0001}

In [ ]:
#D.3.2 Coefficients obtenus par le "meilleur" modèle (ie. celui avec les "meilleurs hyperparamètres"):

In [64]:
EN_best=ElasticNet(alpha=0.01, l1_ratio=0.6000000000000001 , tol=0.0001)

In [65]:
EN_best.fit(train_dataset_n,train_cible_n)

ElasticNet(alpha=0.01, l1_ratio=0.6000000000000001)

In [66]:
pd.DataFrame(EN_best.coef_, train_dataset_n.columns, columns = ['Coeff'])

,Coeff
YearBuilt,0.207070
PropertyGFATotal,0.024624
ZipCode_New,-0.052389
Campus,0.042774
Multifamily_HR,-0.000000
Multifamily_LR,0.052314
Multifamily_MR,0.000116
NonResidential,-0.201574
BALLARD,-0.018477
CENTRAL,-0.012582


In [ ]:
#D.3.3 Prédiction du meilleur modèle:

In [67]:
EN_best_prediction=EN_best.predict(test_dataset_n)
EN_best_prediction

array([ 1.00523215e-01,  3.77236571e-01, -4.67115660e-02,  3.97740186e-01,
        4.39636966e-01,  5.75524235e-01, -9.67165647e-01,  5.32766110e-01,
        5.72991382e-01,  1.51401443e-01, -8.65209665e-01, -7.61116514e-02,
       -4.10846742e-01,  8.30326636e-01, -1.07656058e-01, -7.82683823e-01,
       -1.12983984e-01, -5.73270379e-01, -6.17171542e-01, -2.10844607e-01,
       -6.36075318e-01, -5.10811508e-02, -3.00698085e-01,  2.78420483e-03,
        6.34464357e-01,  4.48679254e-01,  5.81197821e-01, -4.78762359e-01,
       -4.24621952e-02, -6.20688664e-01, -8.43041735e-01,  2.90660989e-01,
        4.75849853e-01,  9.18611018e-02,  2.96678096e-02,  6.06403746e-01,
       -7.89691432e-02,  3.65884779e-01,  6.55371478e-01,  4.10721587e-01,
       -1.54953753e-01,  5.03053356e-01, -9.56236018e-03,  1.17740997e-01,
        1.19025937e-01,  5.29251402e-01,  1.04508696e-01,  5.41704613e-01,
        5.52920075e-01,  6.34344663e-01,  3.99061372e-01,  1.85733446e-01,
        4.86713412e-01, -

In [ ]:
#D.3.4 Calcul de différentes métriques suite à la prédiction du meilleur modèle.

#Calcul du R^2:

In [68]:
R2_EN_best=r2_score(test_cible_n,EN_best_prediction)
R2_EN_best

0.1437659088071982

In [ ]:
#Calcul du MSE:

In [69]:
MSE_EN_best=mean_squared_error(test_cible_n,EN_best_prediction)

In [ ]:
#Calcul du RMSE:

In [70]:
RMSE_EN_best=math.sqrt(MSE_EN_best)

In [ ]:
#Calcul du MAE:

In [71]:
 MAE_EN_best=mean_absolute_error(test_cible_n,EN_best_prediction)

In [ ]:
#D.3.4 Résultats.

In [72]:
results_EN= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_EN= results_EN.append({'Modèle':'ElasticNet','R^2':R2_EN_best,'MSE':MSE_EN_best,'RMSE':RMSE_EN_best,'MAE':MAE_EN_best,'Temps de calcul (s)':EN_time.seconds},ignore_index=True)
results_EN

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,ElasticNet,0.143766,0.883107,0.939738,0.668716,15


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1, mais plus proche que celle du dummy regressor.
# -la valeur du RMSE est supérieure à 0.5, mais moins que le dummy regressor.
# -La valeur du MAE est meilleure que celles du modèle naïf de dummy regressor et de la régression linéaire (ie. plus petite)

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle, qui est quand même meilleur que le 
#  dummy regressor et proche à la régression linéaire.

#-----------------------------------------------------------------------



#D.4 RandomForestClassifier (Méthode ensembliste de régression non linéaire):

In [73]:
RF=RandomForestRegressor()

In [74]:
RF.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
#Le modèle random forest est:
#un méta estimateur qui adapte un certain nombre de classificateurs d'arbre de décision sur divers sous-échantillons de 
#l'ensemble de données et utilise la moyenne pour améliorer la précision prédictive et contrôler le sur ajustement.

#Les hyperparamètres intéressant à être modifiés sont:

# -n_estimators:= Le nombre d'arbres.

# -min_samples_leaf:= Le nombre minimum d'échantillons aux feuille de l'arbre.

# -max_features:= Le nombre de variables (ie. features) à considérer lors de la recherche de la meilleure répartition.

# -max_depth:= il module la profondeur pour éviter que le modèle sur apprenne.



#D.4.0 Optimisation des hyperparamètres.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [75]:
start_time = datetime.now()

In [ ]:
#On essaye plusieurs combinaisons pour ces hyperparamètres, afin de comparer les temps de camculs et les métriques.

In [76]:
#parameters = {'n_estimators' : [10,50,100,300,500],'min_samples_leaf' : [1,3,5,10], 'max_features': ['None','auto', 'sqrt']}
#parameters = {'n_estimators' : [10,50,100,300,500],'max_depth' : [2,3,5,10,15,20,25,30,40,50 ,60], 'max_features': ['None','auto', 'sqrt']}
#parameters = {'n_estimators' : [10,50,100,300,500],'min_samples_leaf' : [1,3,5,10], 'max_features': ['None','auto', 'sqrt'], 'max_depth' : [2,3,5,10,15,20,30,40,50,60]}
parameters = {'n_estimators' : [10,50,100,300,500],'min_samples_leaf' : [1,3,5,10],'max_depth' : [2,3,5,10,20,40,60], 'max_features': ['sqrt']}


RF = GridSearchCV(estimator = RandomForestRegressor(), param_grid = parameters, cv=5, verbose=False)

In [ ]:
#Notez que dans le GridSearchCV 'cv' c'est le nombre de plis (ie. d'itérations).

#D.4.1 Entraînement du modèle. 

In [77]:
RF.fit(train_dataset_n,train_cible_n)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 3, 5, 10, 20, 40, 60],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 3, 5, 10],
                         'n_estimators': [10, 50, 100, 300, 500]},
             verbose=False)

In [78]:
end_time = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [79]:
RF_time = end_time - start_time

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [80]:
RF.score(train_dataset_n,train_cible_n)

0.8731566232090515

In [ ]:
#Les meilleurs hyperparamètres.

In [81]:
RF.best_params_

{'max_depth': 60,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'n_estimators': 300}

In [ ]:
#D.4.2 Importance des variables ("features importances") pour le "meilleur" modèle (ie. celui avec les "meilleurs 
#hyperparamètres"):

In [82]:
RF_best=RandomForestRegressor(max_features='sqrt', min_samples_leaf=1, n_estimators=300, max_depth=60)

In [83]:
RF_best.fit(train_dataset_n,train_cible_n)

RandomForestRegressor(max_depth=40, max_features='sqrt', n_estimators=300)

In [84]:
pd.DataFrame(RF_best.feature_importances_, train_dataset_n.columns, columns = ['Coeff'])

,Coeff
YearBuilt,0.320537
PropertyGFATotal,0.395080
ZipCode_New,0.099425
Campus,0.003465
Multifamily_HR,0.013831
Multifamily_LR,0.007846
Multifamily_MR,0.000040
NonResidential,0.037574
BALLARD,0.009580
CENTRAL,0.009382


In [ ]:
#D.4.3 Prédiction.

In [85]:
RF_best_prediction=RF_best.predict(test_dataset_n)
RF_best_prediction

array([ 3.76919279e-01,  9.17565583e-01,  3.17812736e-01,  5.18187202e-01,
       -5.78791062e-02,  7.53801055e-01, -1.43288910e+00,  3.82290310e-01,
        4.11961696e-01,  2.23137095e-01, -1.55233458e+00, -2.55481124e-01,
       -3.37334765e+00,  6.95462184e-01,  1.08932665e+00, -6.58667241e-01,
       -1.87472680e-01, -4.68951039e-01, -6.67982139e-01,  1.38829196e-01,
       -4.75943618e-01, -6.21957708e-02,  2.60789835e-01,  9.75273532e-03,
        5.12113926e-01, -9.96017079e-01,  1.36411753e+00, -3.65941694e-01,
       -3.38971476e-02, -9.39286638e-01, -8.19229207e-01,  1.50669499e+00,
        3.23024880e-01,  2.67829327e-01,  5.30049868e-01,  1.26752918e+00,
        1.71794337e-01,  2.31015954e-01,  2.36595684e-01,  7.71729422e-01,
       -1.37372814e+00,  7.95680223e-01,  4.02887959e-01,  4.81343401e-02,
       -2.09099213e-01,  5.13779158e-01,  3.96546730e-01,  1.07205655e+00,
        6.01414637e-01,  2.19957048e-01,  8.68303539e-01,  2.40983301e-01,
        8.25784036e-01, -

In [ ]:
#D.4.4 Calcul de différentes métriques suite à la prédiction du modèle.

#Calcul du R^2:

In [86]:
R2_RF_best=r2_score(test_cible_n,RF_best_prediction)
R2_RF_best

0.5281516935670776

In [ ]:
#Calcul du MSE:

In [87]:
MSE_RF_best=mean_squared_error(test_cible_n,RF_best_prediction)

In [ ]:
#Calcul du RMSE:

In [88]:
RMSE_RF_best=math.sqrt(MSE_RF_best)

In [ ]:
#Calcul du MAE:

In [89]:
 MAE_RF_best=mean_absolute_error(test_cible_n,RF_best_prediction)

In [ ]:
#D.4.4 Résultats.

In [90]:
results_RF= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_RF= results_RF.append({'Modèle':'Random Forest','R^2':R2_RF_best,'MSE':MSE_RF_best,'RMSE':RMSE_RF_best,'MAE':MAE_RF_best,'Temps de calcul (s)':RF_time.seconds},ignore_index=True)
results_RF

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Random Forest,0.528152,0.486657,0.697608,0.400458,316


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1, mais plus proche de cells du dummy regressor, la régression linéaire et le 
#  elasticNet.
# -la valeur du RMSE est supérieure à 0.5, mais moins que le dummy regressor, la régression linéaire et le elasticNet.
# -La valeur du MAE est meilleure que celles du modèle naïf de dummy regressor, de la régression linéaire et du elasticNet 
#  (ie. plus petite).

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle, qui est quand même bien meilleur que le 
#  dummy regressor, la régression linéaire et le elasticNet.

#-----------------------------------------------------------------------



#D.5 SVR (Méthode non linéaire):
    # "Support Vector Regression". Cette méthode cherche à minimiser les "pénalités" (L2 en particulier).
    #SVR nous donne la flexibilité de définir la quantité d'erreur acceptable dans notre modèle et trouvera une ligne 
    #appropriée (ou un hyperplan dans des dimensions plus élevées) pour s'adapter aux données.

In [91]:
SVR_1=SVR()

In [92]:
SVR_1.get_params()

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [ ]:
# Les hyperparamètres intéressant à être modifiés sont:

# -gamma:= Kernel coefficient. Il définit l'étendue de l'influence d'un seul exemple d'entraînement, les valeurs faibles 
#          signifiant "loin" et les valeurs élevées signifiant "proche". Les paramètres gamma peuvent être vus comme 
#          l'inverse du rayon d'influence des échantillons sélectionnés par le modèle comme vecteurs supports.

# -epsilon:= C'est le coefficient associé à la pénalité dans la fonction des pertes (loss fonction).

# -C:= Coefficient de régularisation. Il calibre le modèle et doit être positif pour éviter des over et under fit.
#      Le paramètre compromet la classification correcte des exemples d'apprentissage contre la maximisation de la marge de 
#      la fonction de décision. Pour des valeurs plus grandes de C, une marge plus petite sera acceptée si la fonction de 
#      décision réussit mieux à classer correctement tous les points d'apprentissage. Un C inférieur encouragera une plus 
#      grande marge, donc une fonction de décision plus simple, au détriment de la précision de l'apprentissage.




#D.5.0 Optimisation des hyperparamètres.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [93]:
start_time = datetime.now()

In [94]:
parameters = {'gamma' : [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1],'epsilon' : [0.001, 0.01, 0.1, 1], 'C' : [0.001, 0.01, 0.1, 1, 10]}

SVR_g= GridSearchCV(estimator=SVR_1, param_grid = parameters, cv=5, verbose=False)

In [ ]:
#Notez que dans le GridSearchCV 'cv' c'est le nombre de plis (ie. d'itérations).

#D.5.1Entraînement du modèle. 

In [95]:
SVR_g.fit(train_dataset_n,train_cible_n)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10],
                         'epsilon': [0.001, 0.01, 0.1, 1],
                         'gamma': [1e-08, 1e-07, 1e-06, 1e-05, 0.0001, 0.001,
                                   0.01, 0.1]},
             verbose=False)

In [96]:
end_time = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [97]:
SVR_time = end_time - start_time

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [98]:
SVR_g.score(train_dataset_n,train_cible_n)

0.22213164569844634

In [ ]:
#Les meilleurs hyperparamètres.

In [99]:
SVR_g.best_params_

{'C': 1, 'epsilon': 0.1, 'gamma': 0.1}

In [ ]:
#D.5.2 Développement du meilleur modèle:

In [100]:

SVR_best=SVR(C=1.0, epsilon=0.1, gamma=0.1)

In [101]:
SVR_best.fit(train_dataset_n,train_cible_n)

SVR(gamma=0.1)

In [ ]:
#D.5.3 Prédiction.

In [102]:
SVR_best_prediction=SVR_best.predict(test_dataset_n)
SVR_best_prediction

array([ 2.34061859e-01,  4.36705664e-01,  1.50882387e-01,  5.27559306e-01,
        7.18596522e-01,  4.55917823e-01, -4.97903601e-01,  3.29048998e-01,
        5.05416311e-01, -9.53043112e-03, -9.34869118e-01, -4.60503411e-01,
       -4.89397911e-01,  4.25419025e-01,  1.35558788e-01, -5.91169721e-01,
       -1.60723417e-01, -5.38014032e-01, -4.30628797e-01,  2.24241884e-02,
       -4.39256895e-01,  2.01116659e-01,  3.49423077e-01, -3.70424519e-02,
        2.43239631e-01,  4.73159251e-01,  1.42858934e+00, -4.45453414e-01,
       -1.84576883e-01, -9.93162254e-01, -2.27520285e-01,  4.51763669e-01,
        4.28887064e-01,  2.11387284e-01,  1.69866752e-01,  4.28502730e-01,
        8.36528258e-02,  4.56200299e-01,  2.14680731e-01,  6.17689237e-01,
       -3.61925593e-01,  5.12521834e-01,  2.48488515e-01,  2.78187511e-01,
        2.87464695e-01,  3.89575932e-01,  2.63504501e-01,  4.65422288e-01,
        4.66099912e-01,  1.94665347e-01,  1.10429350e+00,  4.99378998e-01,
        1.20806630e+00, -

In [ ]:
#D.5.4 Calcul de différentes métriques suite à la prédiction du modèle.

#Calcul du R^2:

In [103]:
R2_SVR_best=r2_score(test_cible_n,SVR_best_prediction)
R2_SVR_best

0.13301085546886504

In [ ]:
#Calcul du MSE:

In [104]:
MSE_SVR_best=mean_squared_error(test_cible_n,SVR_best_prediction)

In [ ]:
#Calcul du RMSE:

In [105]:
RMSE_SVR_best=math.sqrt(MSE_SVR_best)

In [ ]:
#Calcul du MAE:

In [106]:
 MAE_SVR_best=mean_absolute_error(test_cible_n,SVR_best_prediction)

In [ ]:
#D.5.4 Résultats.

In [107]:
results_SVR= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_SVR= results_SVR.append({'Modèle':'SVR','R^2':R2_SVR_best,'MSE':MSE_SVR_best,'RMSE':RMSE_SVR_best,'MAE':MAE_SVR_best ,'Temps de calcul (s)':SVR_time.seconds},ignore_index=True)
results_SVR

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,SVR,0.133011,0.8942,0.945621,0.638368,342


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1 et de celle du random forest, mais plus proche que celles du dummy regressor, la
#  régression linéaire et le elasticNet.
# -la valeur du RMSE est supérieure à 0.5 et à de celle du random forest, mais plus proche que le dummy regressor, la
#  régression linéaire et le elasticNet.
# -La valeur du MAE est moins bonne celle du random forest, mais meilleure que celles du modèle naïf de dummy regressor, et
#  comparable à celle de la régression linéaire et du elasticNet (ie. plus petite).

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle, qui est quand même bien meilleur que le 
#  dummy regressor, la régression linéaire, le elasticNet et le random forest.


#---------------------------------------------------------------------------------------------------------------------------


#D.6 Tableau résumé:


In [108]:
results_total=np.concatenate((results_DM,results_LR,results_EN,results_RF),axis=1) 
results_total

array([['Dummy Regressor', -0.004197135237299365, 1.0357142114073152,
        1.0177004526909257, 0.735766250370066, 1, 'Linear regression',
        0.1449158351519697, 0.8819212786074929, 0.9391066385706647,
        0.6700131942969035, 1, 'ElasticNet', 0.1437659088071982,
        0.8831072957903343, 0.9397378867483923, 0.6687155172357453, 15,
        'Random Forest', 0.5281516935670776, 0.48665742955497276,
        0.6976083640230906, 0.40045822534675307, 316]], dtype=object)

In [109]:
results_total= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_total= results_total.append({'Modèle':'Dummy Regressor','R^2':R2_DM,'MSE':MSE_DM,'RMSE':RMSE_DM,'MAE':MAE_DM ,'Temps de calcul (s)':DM_time.seconds},ignore_index=True)
results_total= results_total.append({'Modèle':'Linear Regression','R^2':R2_LR,'MSE':MSE_LR,'RMSE':RMSE_LR,'MAE':MAE_LR ,'Temps de calcul (s)':LR_time.seconds},ignore_index=True)
results_total= results_total.append({'Modèle':'ElasticNet','R^2':R2_EN_best,'MSE':MSE_EN_best,'RMSE':RMSE_EN_best,'MAE':MAE_EN_best ,'Temps de calcul (s)':EN_time.seconds},ignore_index=True)
results_total= results_total.append({'Modèle':'Random Forest','R^2':R2_RF_best,'MSE':MSE_RF_best,'RMSE':RMSE_RF_best,'MAE':MAE_RF_best ,'Temps de calcul (s)':RF_time.seconds},ignore_index=True)
results_total= results_total.append({'Modèle':'SVR','R^2':R2_SVR_best,'MSE':MSE_SVR_best,'RMSE':RMSE_SVR_best,'MAE':MAE_SVR_best ,'Temps de calcul (s)':SVR_time.seconds},ignore_index=True)
results_total

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Dummy Regressor,-0.004197,1.035714,1.017700,0.735766,1
1,Linear Regression,0.144916,0.881921,0.939107,0.670013,1
2,ElasticNet,0.143766,0.883107,0.939738,0.668716,15
3,Random Forest,0.528152,0.486657,0.697608,0.400458,316
4,SVR,0.133011,0.894200,0.945621,0.638368,342


In [ ]:
# On regarde que le meilleur modèle est de loin le Random Forest avec:
# -un temps de calcul (ie. temps d'entraînement) d'un peu moins de 7.5 min, 
# -un R^2 qui reste plus petit que 1 mais qui déjà dépasse les 0.5 (On rappelle que à partir de 0.6 on considère que le R^2
#  a une valeur robuste),
# -la MAE la plus petite de tous les modèles,
# -une RMSE bien plus petite que pour les autres modèles, même si elle reste supérieure à 0.5 (pour rappel, des valeurs de 
#  RMSE comprises entre 0.2 et 0.5 montrent que le modèle peut prédire relativement les données avec précision).

#Notez aussi qu'on perçoit que le R^2 n'est pas forcément le meilleure métrique. En fait on sait bien qu'un "mauvais" R^2 ne
#signifie pas que le modèle est forcément mauvais ou inapproprié, on peut avoir des modèles avec mauvais R^2 mais qui 
#restent performants et bon. C'est pour cela que l'on préfère considérer aussi le RMSE et le MAE pour valider nos modèles.


#---------------------------------------------------------------------------------------------------------------------------
#---------------------------------------------------------------------------------------------------------------------------


#Faisons maintemant la même chose mais en considérant parmi nos varibles descriptives (ie.features) la variable 
#ENERGYSTARScore

In [ ]:
#---

In [ ]:
#---

In [ ]:
#A. IMPORTER LES DONNÉES:

In [110]:
df_E=pd.read_csv('Data_clean_E.csv', sep=',', low_memory=False)

In [111]:
df_E.count()

OSEBuildingID                    2172
TaxParcelIdentificationNumber    2172
PropertyName                     2172
SourceEUI_kBtu_sf_log            2172
TotalGHGEmissions_log            2172
BuildingType                     2172
YearBuilt                        2172
PropertyGFATotal                 2172
Neighborhood                     2172
ZipCode_New                      2172
ENERGYSTARScore                  2172
dtype: int64

In [112]:
encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)

In [ ]:
#B.1.1 'BuildingType':

In [113]:
encoder_BuildingType_E = pd.DataFrame(encoder.fit_transform(df_E[['BuildingType']]))
encoder_BuildingType_E

,0,1,2,3
0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0
...,...,...,...,...
2167,0.0,0.0,1.0,0.0
2168,0.0,1.0,0.0,0.0
2169,0.0,1.0,0.0,0.0
2170,0.0,1.0,0.0,0.0


In [114]:
encoder_BuildingType_E.rename(columns={0:'Campus',1:'Multifamily_HR',2:'Multifamily_LR',3:'Multifamily_MR', 4:'NonResidential', 5:'NonResidential_COS', 6:'NonResidential_WA', 7:'SPS-District_K-12'},inplace = True)
encoder_BuildingType_E

,Campus,Multifamily_HR,Multifamily_LR,Multifamily_MR
0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0
...,...,...,...,...
2167,0.0,0.0,1.0,0.0
2168,0.0,1.0,0.0,0.0
2169,0.0,1.0,0.0,0.0
2170,0.0,1.0,0.0,0.0


In [ ]:
#B.1.2 'Neighborhood':

In [115]:
encoder_Neighborhood_E = pd.DataFrame(encoder.fit_transform(df_E[['Neighborhood']]))
encoder_Neighborhood_E

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2167,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2168,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2169,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2170,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
encoder_Neighborhood_E.rename(columns={0:'BALLARD',1:'CENTRAL',2:'DELRIDGE',3:'DOWNTOWN', 4:'EAST', 5:'GREATER_DUWAMISH', 6:'LAKE_UNION', 7:'MAGNOLIA_QUEEN_ANNE', 8:'NORTH', 9:'NORTHEAST', 10:'NORTHWEST', 11:'SOUTHEAST', 12:'SOUTHWEST' },inplace = True)
encoder_Neighborhood_E

,BALLARD,CENTRAL,DELRIDGE,DOWNTOWN,EAST,GREATER_DUWAMISH,LAKE_UNION,MAGNOLIA_QUEEN_ANNE,NORTH,NORTHEAST,NORTHWEST,SOUTHEAST,SOUTHWEST
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2167,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2168,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2169,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2170,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#B.2 On fusionne les colonnes résultant du OneHotEncoder dans le tableau initial.

In [117]:
f1_df_E = df_E.join(encoder_BuildingType_E)

In [118]:
final_df_E= f1_df_E.join(encoder_Neighborhood_E)
final_df_E.count()

OSEBuildingID                    2172
TaxParcelIdentificationNumber    2172
PropertyName                     2172
SourceEUI_kBtu_sf_log            2172
TotalGHGEmissions_log            2172
BuildingType                     2172
YearBuilt                        2172
PropertyGFATotal                 2172
Neighborhood                     2172
ZipCode_New                      2172
ENERGYSTARScore                  2172
Campus                           2172
Multifamily_HR                   2172
Multifamily_LR                   2172
Multifamily_MR                   2172
BALLARD                          2172
CENTRAL                          2172
DELRIDGE                         2172
DOWNTOWN                         2172
EAST                             2172
GREATER_DUWAMISH                 2172
LAKE_UNION                       2172
MAGNOLIA_QUEEN_ANNE              2172
NORTH                            2172
NORTHEAST                        2172
NORTHWEST                        2172
SOUTHEAST   

In [ ]:
#--------------------------------------------------------------------------------------------------------------------------

#C.Le dataset.

#C.1 Élimination des variables non nécessaires.

#On élimine les colonnes des variables catégorielles initiales et celles d'identification des bâtiments:
#'OSEBuildingID','TaxParcelIdentificationNumber','PropertyName','BuildingType','Neighborhood'.
#On reste aussi uniquement avec la variable cible qui nous intéresse: 'SourceEUI_kBtu_sf_log'.

In [119]:
final_df_E.drop({'TotalGHGEmissions_log','OSEBuildingID','TaxParcelIdentificationNumber','PropertyName','BuildingType','Neighborhood'}, axis=1, inplace=True)

In [ ]:
#C.2 Le dataset.

In [120]:
dataset_E=final_df_E
dataset_E.head(5)

,SourceEUI_kBtu_sf_log,YearBuilt,PropertyGFATotal,ZipCode_New,ENERGYSTARScore,Campus,Multifamily_HR,Multifamily_LR,Multifamily_MR,BALLARD,...,DOWNTOWN,EAST,GREATER_DUWAMISH,LAKE_UNION,MAGNOLIA_QUEEN_ANNE,NORTH,NORTHEAST,NORTHWEST,SOUTHEAST,SOUTHWEST
0,5.161925,1927,88434,98101,65.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5.259057,1996,103566,98101,51.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5.495938,1969,961990,98101,18.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.437209,1980,119890,98121,67.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5.013963,1926,83008,98101,25.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#C.3 Training et Test sets.

#On divise les données en 2 sets: un "training set" et un "test set".

In [121]:
train_dataset_E = dataset_E.sample(frac=0.8,random_state=0)
test_dataset_E = dataset_E.drop(train_dataset_E.index)
train_dataset_E

,SourceEUI_kBtu_sf_log,YearBuilt,PropertyGFATotal,ZipCode_New,ENERGYSTARScore,Campus,Multifamily_HR,Multifamily_LR,Multifamily_MR,BALLARD,...,DOWNTOWN,EAST,GREATER_DUWAMISH,LAKE_UNION,MAGNOLIA_QUEEN_ANNE,NORTH,NORTHEAST,NORTHWEST,SOUTHEAST,SOUTHWEST
194,5.066385,1916,350310,98121,89.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
303,4.254193,1991,323916,98108,81.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1205,4.582925,2006,269297,98115,80.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1695,4.584967,1972,48112,98133,73.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1981,4.334673,1962,51410,98116,87.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2157,5.910526,1920,374466,98101,74.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1843,5.173321,1984,57110,98119,69.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1932,5.031744,1970,44000,98108,15.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1556,4.730921,1965,27800,98108,13.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
train_dataset_E.sum()

SourceEUI_kBtu_sf_log    8.398151e+03
YearBuilt                3.413324e+06
PropertyGFATotal         2.254345e+08
ZipCode_New              1.705263e+08
ENERGYSTARScore          1.122280e+05
Campus                   7.000000e+00
Multifamily_HR           1.580000e+03
Multifamily_LR           1.400000e+01
Multifamily_MR           1.370000e+02
BALLARD                  7.100000e+01
CENTRAL                  5.200000e+01
DELRIDGE                 4.600000e+01
DOWNTOWN                 4.070000e+02
EAST                     1.140000e+02
GREATER_DUWAMISH         3.400000e+02
LAKE_UNION               1.650000e+02
MAGNOLIA_QUEEN_ANNE      1.510000e+02
NORTH                    6.800000e+01
NORTHEAST                1.330000e+02
NORTHWEST                9.100000e+01
SOUTHEAST                5.900000e+01
SOUTHWEST                4.100000e+01
dtype: float64

In [ ]:
#C.4 Quelques statistiques.

In [123]:
train_stats_E = train_dataset_E.describe()
train_stats_E = train_stats_E.transpose()
train_stats_E

,count,mean,std,min,25%,50%,75%,max
SourceEUI_kBtu_sf_log,1738.0,4.832077,0.758092,1.098612,4.383899,4.882802,5.271845,7.871311e+00
YearBuilt,1738.0,1963.937860,31.841317,1900.000000,1940.000000,1968.000000,1989.000000,2.015000e+03
PropertyGFATotal,1738.0,129709.138665,212468.864690,16300.000000,32975.250000,55279.500000,124472.000000,1.952220e+06
ZipCode_New,1738.0,98116.404488,17.296471,98020.000000,98104.000000,98109.000000,98122.000000,9.819900e+04
ENERGYSTARScore,1738.0,64.573072,28.440620,1.000000,48.000000,72.000000,89.000000,1.000000e+02
Campus,1738.0,0.004028,0.063354,0.000000,0.000000,0.000000,0.000000,1.000000e+00
Multifamily_HR,1738.0,0.909091,0.287563,0.000000,1.000000,1.000000,1.000000,1.000000e+00
Multifamily_LR,1738.0,0.008055,0.089414,0.000000,0.000000,0.000000,0.000000,1.000000e+00
Multifamily_MR,1738.0,0.078826,0.269545,0.000000,0.000000,0.000000,0.000000,1.000000e+00
BALLARD,1738.0,0.040852,0.198003,0.000000,0.000000,0.000000,0.000000,1.000000e+00


In [ ]:
#C.5 Normalisation des données.

#Il est recommandé de normaliser les variables qui ont différentes échelles (cf. statistiques en haut). En fait, même si 
#le modèle peut converger sans normalisation ça rend l'entraînement plus difficile et le résultat dépendant des unités 
#utilisées pour les variables.

#Notez que nous avons générer les statistiques précédentes uniquement sur le "set training", ces statistiques sont également
#utilisées pour normaliser l'ensemble de données de test, ceci l'ensemble de données de test doivent être projetées dans la 
#même distribution que celle sur laquelle le training fut fait.

#On a décidé de normaliser aussi la variable cible. En fait même si ceci n'est pas vraiment nécessaire, c'est juste que des 
#fois si on ne normalise pas la variable cible il peut y avoir des bias et on a préféré d'éviter ceci depuis le début.

In [124]:
def norm(x):
  return (x - train_stats_E['mean']) / train_stats_E['std']
train_dataset_En = norm(train_dataset_E)
test_dataset_En = norm(test_dataset_E)

In [125]:
train_stats_En = train_dataset_En.describe()
train_stats_En = train_stats_En.transpose()
train_stats_En

,count,mean,std,min,25%,50%,75%,max
SourceEUI_kBtu_sf_log,1738.0,-6.374775e-15,1.0,-4.924816,-0.591192,0.066911,0.580098,4.009055
YearBuilt,1738.0,2.292246e-15,1.0,-2.008016,-0.751786,0.127575,0.787095,1.603644
PropertyGFATotal,1738.0,1.065987e-18,1.0,-0.533768,-0.455285,-0.350308,-0.024649,8.577779
ZipCode_New,1738.0,2.672597e-13,1.0,-5.573651,-0.717169,-0.428092,0.323506,4.775281
ENERGYSTARScore,1738.0,-2.065698e-16,1.0,-2.235291,-0.582725,0.261138,0.858875,1.245645
Campus,1738.0,3.230379e-16,1.0,-0.063573,-0.063573,-0.063573,-0.063573,15.720796
Multifamily_HR,1738.0,-1.583569e-16,1.0,-3.161368,0.316137,0.316137,0.316137,0.316137
Multifamily_LR,1738.0,-1.182582e-15,1.0,-0.090089,-0.090089,-0.090089,-0.090089,11.093782
Multifamily_MR,1738.0,8.240436e-17,1.0,-0.292442,-0.292442,-0.292442,-0.292442,3.417515
BALLARD,1738.0,-1.244689e-16,1.0,-0.206318,-0.206318,-0.206318,-0.206318,4.844106


In [ ]:
#C.6 Séparez la variable cible des variables prédictives. 

#La variable cible est la valeur que l'on entraînera le modèle à prédire.

In [126]:
train_cible_En = train_dataset_En.pop('SourceEUI_kBtu_sf_log')
test_cible_En = test_dataset_En.pop('SourceEUI_kBtu_sf_log')

In [ ]:
#--------------------------------------------------------------------------------------------------------------------------

#D. Les Modèles.
#Construction et entraînement des modèles.
    #Notez que la fonction ".fit", c'est celle qui est chargée d'entraîner notre modèle et que la fonction ".predict", c'est
    #celle qui est chargée de prédire la variable cible à partir du dataset test.
    
    
#------

#D.1 DummyRegressor: 
    #On prend la médiane car elle est statistiquement préférable à la moyenne.
    #Sans optimisation des hyperparamètres (pour le DummyRegressor il n'y a pas vraiment des paramètres à optimiser).

In [127]:
DM_E= DummyRegressor(strategy='median')

In [ ]:
#D.1.1 Entraînement du modèle.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [128]:
start_time_E = datetime.now()

In [129]:
DM_E.fit(train_dataset_En,train_cible_En)

DummyRegressor(strategy='median')

In [130]:
end_time_E = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [131]:
DM_time_E = end_time_E - start_time_E

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [132]:
DM_E.score(train_dataset_En,train_cible_En)

-0.004479610196447492

In [ ]:
#Notez que la valeur absolue du R^2 (qui est donné par le "score") est loin de 1, ce qui montre la faiblesse de ce 
#modèle.

#D.1.2 Prédiction.

In [133]:
DM_prediction_E=DM_E.predict(test_dataset_En)
DM_prediction_E

array([0.06691063, 0.06691063, 0.06691063, 0.06691063, 0.06691063,
       0.06691063, 0.06691063, 0.06691063, 0.06691063, 0.06691063,
       0.06691063, 0.06691063, 0.06691063, 0.06691063, 0.06691063,
       0.06691063, 0.06691063, 0.06691063, 0.06691063, 0.06691063,
       0.06691063, 0.06691063, 0.06691063, 0.06691063, 0.06691063,
       0.06691063, 0.06691063, 0.06691063, 0.06691063, 0.06691063,
       0.06691063, 0.06691063, 0.06691063, 0.06691063, 0.06691063,
       0.06691063, 0.06691063, 0.06691063, 0.06691063, 0.06691063,
       0.06691063, 0.06691063, 0.06691063, 0.06691063, 0.06691063,
       0.06691063, 0.06691063, 0.06691063, 0.06691063, 0.06691063,
       0.06691063, 0.06691063, 0.06691063, 0.06691063, 0.06691063,
       0.06691063, 0.06691063, 0.06691063, 0.06691063, 0.06691063,
       0.06691063, 0.06691063, 0.06691063, 0.06691063, 0.06691063,
       0.06691063, 0.06691063, 0.06691063, 0.06691063, 0.06691063,
       0.06691063, 0.06691063, 0.06691063, 0.06691063, 0.06691

In [ ]:
#D.1.3 Calcul de différentes métriques suite à la prédiction du modèle.

#Calcul du R^2:

In [134]:
R2_DM_E=r2_score(test_cible_En,DM_prediction_E)
R2_DM_E

-0.06418488379077725

In [ ]:
#Calcul du MSE:

In [135]:
MSE_DM_E=mean_squared_error(test_cible_En,DM_prediction_E)

In [ ]:
#Calcul du RMSE:

In [136]:
RMSE_DM_E=math.sqrt(MSE_DM_E)

In [ ]:
#Calcul du MAE:

In [137]:
 MAE_DM_E=mean_absolute_error(test_cible_En,DM_prediction_E)

In [138]:
#D.1.5 Résultats.

In [139]:
results_DM_E= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_DM_E= results_DM_E.append({'Modèle':'Dummy Regressor','R^2':R2_DM_E,'MSE':MSE_DM_E,'RMSE':RMSE_DM_E,'MAE':MAE_DM_E ,'Temps de calcul (s)':DM_time_E.seconds},ignore_index=True)
results_DM_E

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Dummy Regressor,-0.064185,1.166623,1.080103,0.777485,0


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1.
# -la valeur du RMSE est supérieure à 0.5 (des valeurs de RMSE comprises entre 0.2 et 0.5 montrent que le modèle peut 
#  prédire relativement les données avec précision) et
# -La valeur du MAE de modèle naïf de Random forest servira de base, on rappelle qu'il n'y a pas de valeur correcte pour la 
#  MSE. En termes simples, plus la valeur est faible, mieux c'est et 0 signifie que le modèle est parfait. On se base dans
#  la MEA d'un modèle naïf (comme le dummy regressor) pour comparer avec la MEA de modèles plus performants.

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle.

# Notez qu'en considérant le ENERGYSTARScore on n'améliore pas les valeurs des métriques pour le dummy regressor.
#-----------------------------------------------------------------------

#D.2 LinearRegression.
    #Sans optimisation des hyperparamètres (pour la régression linéaire il n'y a pas vraiment des paramètres à optimiser).

In [140]:
LR_E=LinearRegression()

In [141]:
LR_E.get_params()

{'copy_X': True,
 'fit_intercept': True,
 'n_jobs': None,
 'normalize': False,
 'positive': False}

In [ ]:
#D.2.1 Entraînement du modèle.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [142]:
start_time_E = datetime.now()

In [143]:
LR_E.fit(train_dataset_En,train_cible_En)

LinearRegression()

In [144]:
end_time_E = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [145]:
LR_time_E = end_time_E - start_time_E

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [146]:
LR_E.score(train_dataset_En,train_cible_En)

0.36672600208956785

In [ ]:
#D.2.2 Prédiction.

In [147]:
LR_prediction_E=LR_E.predict(test_dataset_En)
LR_prediction_E

array([ 1.26534189, -0.36161124, -0.55643546,  0.98018564, -0.08622061,
        0.11104501, -0.24247061,  1.06563485, -0.14554679,  0.04854501,
        0.45674814,  0.05049814, -0.05936515, -0.34867179, -0.97098624,
        0.26534189, -0.52372061,  0.10713876, -1.45926749,  0.51143564,
       -0.02518546, -0.5163964 ,  0.87276376,  1.28487314,  1.03877939,
        0.09346689,  0.41377939,  0.06612314,  0.18331064, -0.26200186,
       -0.04325186,  0.43331064, -0.29715811,  0.99190439, -0.49637686,
        0.48799814, -0.04374015,  0.38252939,  0.35127939,  0.00752939,
        0.06221689,  0.63252939, -0.12137686,  0.06612314,  0.37081064,
       -0.03153311,  0.88838876,  0.27120126, -0.28739249, -0.06082999,
        0.10323251,  0.33370126, -0.02567374, -0.86747061, -0.24051749,
        0.28487314, -0.53934561, -0.35624015,  0.13057626,  1.22823251,
        0.05635751,  0.51729501, -0.66043936, -0.47879874, -0.49247061,
       -0.25809561, -1.28739249, -1.02762686,  0.55831064,  0.04

In [ ]:
#D.2.3 Calcul de différentes métriques suite à la prédiction du modèle.

#Calcul du R^2:

In [148]:
R2_LR_E=r2_score(test_cible_En,LR_prediction_E)
R2_LR_E

0.32989145601937686

In [ ]:
#Calcul du MSE:

In [149]:
MSE_LR_E=mean_squared_error(test_cible_En,LR_prediction_E)

In [ ]:
#Calcul du RMSE:

In [150]:
RMSE_LR_E=math.sqrt(MSE_LR_E)

In [ ]:
#Calcul du MAE:

In [151]:
 MAE_LR_E=mean_absolute_error(test_cible_En,LR_prediction_E)

In [ ]:
#D.2.5 Résultats.

In [152]:
results_LR_E= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_LR_E= results_LR_E.append({'Modèle':'Linear regression','R^2':R2_LR_E,'MSE':MSE_LR_E,'RMSE':RMSE_LR_E,'MAE':MAE_LR_E,'Temps de calcul (s)':LR_time_E.seconds},ignore_index=True)
results_LR_E

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Linear regression,0.329891,0.734613,0.857096,0.559039,0


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1, mais plus proche que celle du dummy regressor.
# -la valeur du RMSE est supérieure à 0.5, mais moins que le dummy regressor.
# -La valeur du MAE est meilleure que celle du modèle naïf de dummy regressor(ie. plus petite).

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle, qui est quand même meilleur que le 
#  dummy regressor.

# Notez qu'en considérant le ENERGYSTARScore on améliore un peu les valeurs des métriques pour la linear regression.
# mais vaut-il vraiment le coup pour cette amélioration de considérer cette variable qui a tellement de données manquantes et qui est difficile à calculer?

#D.2.6 Coefficients obtenus par le modèle:

In [153]:
pd.DataFrame(LR_E.coef_, train_dataset_En.columns, columns = ['Coeff'])

,Coeff
YearBuilt,1.874269e-01
PropertyGFATotal,9.461490e-02
ZipCode_New,3.353485e-03
ENERGYSTARScore,-4.921162e-01
Campus,-6.292907e+12
Multifamily_HR,-2.856346e+13
Multifamily_LR,-8.881502e+12
Multifamily_MR,-2.677378e+13
BALLARD,-6.777192e+12
CENTRAL,-5.832883e+12


In [ ]:
#-----------------------------------------------------------------------

#D.3 ElasticNet (méthode de régression linéaire).

    #C'est une extension de la régression linéaire, avec un paramètre de pénalité supplémentaire qui vise à minimiser la 
    #complexité et/ou à réduire le nombre de variables (features) utilisées dans le modèle final, ce qui rend intéressant 
    #le fait de le tester. Quoi qu'il en soit, 'objectif -comme pour de nombreux modèles- est de réduire l'erreur de 
    #l'ensemble de test.
    #ElasticNet est un mélange de Lasso (qui inclu la pénalité L1) et Ridge Regression (qui inclue la pénalité L2).

In [154]:
EN_E=ElasticNet()

In [155]:
EN_E.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'l1_ratio': 0.5,
 'max_iter': 1000,
 'normalize': False,
 'positive': False,
 'precompute': False,
 'random_state': None,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

In [ ]:
#Le modèle ElasticNet est:
#(1 / (2 * n_samples) )* ||y - Xw||^2_2 + alpha * l1_ratio * ||w||_1 + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2

#Les hyperparamètres intéressant à être modifiés sont:

# -alpha:= attribue le poids accordé à chacune des pénalités L1 et L2.

# -l1_ratio:= C'est un paramètre de pénalité de mélange. Pour l1_ratio = 0 la pénalité est une pénalité L2 (Ridge). 
#             Pour l1_ratio = 1 c'est une pénalité L1 (Lasso). Pour 0 < l1_ratio < 1, la pénalité est une combinaison de L1 
#             et L2.

# -tol:= La tolérance d'optimisation, ie. quand le modèle arrête-t-il le processus d'optimisation.



#D.3.0 Optimisation des hyperparamètres.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [156]:
start_time_E = datetime.now()

In [157]:
parameters = {'tol' : [0.1,0.01,0.001,0.0001], 'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100], 'l1_ratio': np.arange(0.0, 1.0, 0.1)} #0: Ridge; 1:Lasso

EN_E = GridSearchCV(estimator = ElasticNet(), param_grid = parameters, cv=5, verbose=False)

In [ ]:
#Notez que dans le GridSearchCV 'cv' c'est le nombre de plis (ie. d'itérations).

#D.3.1 Entraînement du modèle.

In [158]:
EN_E.fit(train_dataset_En,train_cible_En)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 460.12133356308635, tolerance: 143.5279217218734
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 435.4576825426976, tolerance: 137.5302579500727
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 433.34397982116684, tolerance: 138.47421915343918
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWar

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23.659083561315242, tolerance: 0.1382435857361657
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 20.886319115262268, tolerance: 14.352792172187343
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14.794142677263721, tolerance: 13.75302579500727
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceW

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 14.331410045297162, tolerance: 13.69079124917884
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.661159404900104, tolerance: 1.4352792172187343
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.047806323553004, tolerance: 1.3847421915343918
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWar

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.838254584911283, tolerance: 1.3824358573616569
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.5313125640684575, tolerance: 0.1375302579500727
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.7367166914244763, tolerance: 0.1384742191534392
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceW

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 432.67923380580834, tolerance: 0.1382435857361657
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.1803173115847585, tolerance: 0.14352792172187342
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.15295637524843642, tolerance: 0.1375302579500727
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: Convergen

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 483.8915921730152, tolerance: 143.5279217218734
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 458.48394201194725, tolerance: 137.5302579500727
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 457.1645772267337, tolerance: 138.47421915343918
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarn

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 558.1880274772828, tolerance: 1.3824358573616569
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 586.466784271464, tolerance: 0.14352792172187342
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 558.7184794365513, tolerance: 0.1375302579500727
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWar

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 689.4539795765045, tolerance: 13.847421915343919
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 681.7607885860222, tolerance: 13.69079124917884
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 688.2499138367298, tolerance: 13.82435857361657
  model = cd_fast.enet_coordinate_descent(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarni

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                         'l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                         'tol': [0.1, 0.01, 0.001, 0.0001]},
             verbose=False)

In [159]:
end_time_E = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [160]:
EN_time_E = end_time_E - start_time_E

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [161]:
EN_E.score(train_dataset_En,train_cible_En)

0.3663798154476088

In [ ]:
#Les meilleurs hyperparamètres.

In [162]:
EN_E.best_params_

{'alpha': 0.01, 'l1_ratio': 0.9, 'tol': 0.1}

In [ ]:
#D.3.2 Coefficients obtenus par le "meilleur" modèle (ie. celui avec les "meilleurs hyperparamètres"):

In [164]:
EN_best_E=ElasticNet(alpha=0.01, l1_ratio=0.9 , tol=0.1)

In [165]:
EN_best_E.fit(train_dataset_En,train_cible_En)

ElasticNet(alpha=0.01, l1_ratio=0.9, tol=0.1)

In [166]:
pd.DataFrame(EN_best_E.coef_, train_dataset_En.columns, columns = ['Coeff'])

,Coeff
YearBuilt,0.182858
PropertyGFATotal,0.087053
ZipCode_New,0.000000
ENERGYSTARScore,-0.476580
Campus,-0.000000
Multifamily_HR,0.035353
Multifamily_LR,0.028035
Multifamily_MR,-0.016914
BALLARD,-0.030016
CENTRAL,-0.013303


In [ ]:
#D.3.3 Prédiction du meilleur modèle:

In [167]:
EN_best_prediction_E=EN_best_E.predict(test_dataset_En)
EN_best_prediction_E

array([ 1.21876765, -0.44507943, -0.60209295,  0.92852841, -0.08596704,
        0.10577041, -0.11595244,  1.07730015, -0.16722129, -0.03191242,
        0.47896125,  0.0945508 , -0.01220037, -0.36113569, -0.93836558,
        0.29469479, -0.54853265,  0.10017064, -1.47775414,  0.60227557,
       -0.09127892, -0.52871697,  0.84172444,  1.23827028,  0.99624889,
        0.08741931,  0.38356736,  0.0559838 ,  0.1706629 , -0.26250096,
       -0.04622393,  0.40418755, -0.29617881,  0.74655462, -0.48794415,
        0.45867451, -0.00508597,  0.38118017,  0.33098409,  0.00335561,
        0.05544025,  0.60890217, -0.12347606,  0.05786825,  0.35701868,
       -0.05014194,  0.85613681,  0.25771574, -0.27726816, -0.05714504,
        0.09775841,  0.32381757, -0.03082901, -0.83380217, -0.23178312,
        0.27479421, -0.50979682, -0.30044975,  0.13160603,  1.1872141 ,
        0.05331107,  0.49837271, -0.63348191, -0.46870911, -0.48092203,
       -0.25335684, -1.24420012, -0.98816546,  0.53630733,  0.05

In [ ]:
#D.3.4 Calcul de différentes métriques suite à la prédiction du meilleur modèle.

#Calcul du R^2:

In [168]:
R2_EN_best_E=r2_score(test_cible_En,EN_best_prediction_E)
R2_EN_best_E

0.3294162687694866

In [ ]:
#Calcul du MSE:

In [169]:
MSE_EN_best_E=mean_squared_error(test_cible_En,EN_best_prediction_E)

In [ ]:
#Calcul du RMSE:

In [170]:
RMSE_EN_best_E=math.sqrt(MSE_EN_best_E)

In [ ]:
#Calcul du MAE:

In [171]:
 MAE_EN_best_E=mean_absolute_error(test_cible_En,EN_best_prediction_E)

In [ ]:
#D.3.4 Résultats.

In [172]:
results_EN_E= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_EN_E= results_EN_E.append({'Modèle':'ElasticNet','R^2':R2_EN_best_E,'MSE':MSE_EN_best_E,'RMSE':RMSE_EN_best_E,'MAE':MAE_EN_best_E,'Temps de calcul (s)':EN_time_E.seconds},ignore_index=True)
results_EN_E

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,ElasticNet,0.329416,0.735134,0.857399,0.559058,15


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1, mais plus proche que celle du dummy regressor.
# -la valeur du RMSE est supérieure à 0.5, mais moins que le dummy regressor.
# -La valeur du MAE est meilleure que celles du modèle naïf de dummy regressor et de la régression linéaire (ie. plus petite)

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle, qui est quand même meilleur que le 
#  dummy regressor et proche à la régression linéaire.


# Notez qu'en considérant le ENERGYSTARScore on améliore un peu les valeurs des métriques, mais vaut-il vraiment le coup pour
#cette amélioration de considérer cette variable qui a tellement de données manquantes et qui est difficile à calculer?

#-----------------------------------------------------------------------



#D.4 RandomForestClassifier (Méthode ensembliste de régression non linéaire):

In [173]:
RF_E=RandomForestRegressor()

In [174]:
RF_E.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [ ]:
#Le modèle random forest est:
#un méta estimateur qui adapte un certain nombre de classificateurs d'arbre de décision sur divers sous-échantillons de 
#l'ensemble de données et utilise la moyenne pour améliorer la précision prédictive et contrôler le sur ajustement.

#Les hyperparamètres intéressant à être modifiés sont:

# -n_estimators:= Le nombre d'arbres.

# -min_samples_leaf:= Le nombre minimum d'échantillons aux feuille de l'arbre.

# -max_features:= Le nombre de variables (ie. features) à considérer lors de la recherche de la meilleure répartition.

# -max_depth:= il module la profondeur pour éviter que le modèle sur apprenne.



#D.4.0 Optimisation des hyperparamètres.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [175]:
start_time_E = datetime.now()

In [ ]:
#On essaye plusieurs combinaisons pour ces hyperparamètres, afin de comparer les temps de camculs et les métriques.

In [176]:
#parameters = {'n_estimators' : [10,50,100,300,500],'min_samples_leaf' : [1,3,5,10], 'max_features': ['None','auto', 'sqrt']}
#parameters = {'n_estimators' : [10,50,100,300,500],'max_depth' : [2,3,5,10,15,20,25,30,40,50 ,60], 'max_features': ['None','auto', 'sqrt']}
#parameters = {'n_estimators' : [10,50,100,300,500],'min_samples_leaf' : [1,3,5,10], 'max_features': ['None','auto', 'sqrt'], 'max_depth' : [2,3,5,10,15,20,30,40,50,60]}
parameters = {'n_estimators' : [10,50,100,300,500],'min_samples_leaf' : [1,3,5,10],'max_depth' : [2,3,5,10,20,40,60], 'max_features': ['sqrt']}


RF_E = GridSearchCV(estimator = RandomForestRegressor(), param_grid = parameters, cv=5, verbose=False)

In [ ]:
#Notez que dans le GridSearchCV 'cv' c'est le nombre de plis (ie. d'itérations).

#D.4.1 Entraînement du modèle. 

In [177]:
RF_E.fit(train_dataset_En,train_cible_En)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [2, 3, 5, 10, 20, 40, 60],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [1, 3, 5, 10],
                         'n_estimators': [10, 50, 100, 300, 500]},
             verbose=False)

In [178]:
end_time_E = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [179]:
RF_time_E = end_time_E - start_time_E

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [180]:
RF_E.score(train_dataset_En,train_cible_En)

0.9555670924347107

In [ ]:
#Les meilleurs hyperparamètres.

In [181]:
RF_E.best_params_

{'max_depth': 40,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'n_estimators': 300}

In [ ]:
#D.4.2 Importance des variables ("features importances") pour le "meilleur" modèle (ie. celui avec les "meilleurs 
#hyperparamètres"):

In [182]:
RF_best_E=RandomForestRegressor(max_features='sqrt', min_samples_leaf=1, n_estimators=300, max_depth=40)

In [183]:
RF_best_E.fit(train_dataset_En,train_cible_En)

RandomForestRegressor(max_depth=40, max_features='sqrt', n_estimators=300)

In [184]:
pd.DataFrame(RF_best_E.feature_importances_, train_dataset_En.columns, columns = ['Coeff'])

,Coeff
YearBuilt,0.192347
PropertyGFATotal,0.213532
ZipCode_New,0.092907
ENERGYSTARScore,0.323350
Campus,0.001136
Multifamily_HR,0.012328
Multifamily_LR,0.002360
Multifamily_MR,0.015099
BALLARD,0.012170
CENTRAL,0.007037


In [ ]:
#D.4.3 Prédiction.

In [185]:
RF_best_prediction_E=RF_best_E.predict(test_dataset_En)
RF_best_prediction_E

array([ 8.12465767e-01, -5.82827511e-01, -9.61516573e-01,  5.17051980e-01,
        3.11330127e-01,  1.76860411e-01,  6.32923441e-02,  9.42102231e-01,
       -3.56453258e-01, -4.91710568e-01,  1.05026900e+00, -9.64627959e-02,
        6.82930312e-01, -6.57889606e-01, -1.04677726e+00,  6.47888268e-01,
       -4.74015753e-01,  2.54682954e-01, -7.63456026e-01,  1.07022813e+00,
       -3.24495992e-01, -5.22691702e-01,  8.86786179e-01,  1.03326277e+00,
        1.00473500e+00,  8.16307578e-02,  3.55862109e-01,  2.27283278e-01,
        4.58791535e-01, -2.71974259e-01,  7.28045129e-02,  5.00624115e-01,
        7.98544950e-02,  5.16826598e-01, -4.90863133e-01,  8.10240902e-01,
        2.49563885e-01, -3.90347744e-02,  5.51270825e-01, -1.40045564e-01,
        4.25592180e-01,  7.93146288e-01, -3.40838263e-01,  3.57095303e-01,
        5.11409455e-01,  8.28469048e-03,  1.04269272e+00,  5.33198836e-01,
        2.50322335e-01,  4.49571601e-01,  1.70424388e-01,  7.93541564e-01,
        5.25522575e-01, -

In [ ]:
#D.4.4 Calcul de différentes métriques suite à la prédiction du modèle.

#Calcul du R^2:

In [186]:
R2_RF_best_E=r2_score(test_cible_En,RF_best_prediction_E)
R2_RF_best_E

0.5817750176884506

In [ ]:
#Calcul du MSE:

In [187]:
MSE_RF_best_E=mean_squared_error(test_cible_En,RF_best_prediction_E)

In [ ]:
#Calcul du RMSE:

In [188]:
RMSE_RF_best_E=math.sqrt(MSE_RF_best_E)

In [ ]:
#Calcul du MAE:

In [189]:
 MAE_RF_best_E=mean_absolute_error(test_cible_En,RF_best_prediction_E)

In [ ]:
#D.4.4 Résultats.

In [190]:
results_RF_E= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_RF_E= results_RF_E.append({'Modèle':'Random Forest','R^2':R2_RF_best_E,'MSE':MSE_RF_best_E,'RMSE':RMSE_RF_best_E,'MAE':MAE_RF_best_E,'Temps de calcul (s)':RF_time_E.seconds},ignore_index=True)
results_RF_E

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Random Forest,0.581775,0.458483,0.677114,0.362651,266


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1, mais plus proche de 0.6 et de celles du dummy regressor, la régression 
#  linéaire et le elasticNet.
# -la valeur du RMSE est supérieure à 0.5, mais moins que le dummy regressor, la régression linéaire et le elasticNet.
# -La valeur du MAE est meilleure que celles du modèle naïf de dummy regressor, de la régression linéaire et du elasticNet 
#  (ie. plus petite).

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle, qui est quand même bien meilleur que le 
#  dummy regressor, la régression linéaire et le elasticNet.


# Notez qu'en considérant le ENERGYSTARScore on améliore un peu les valeurs des métriques, mais vaut-il vraiment le coup pour
# cette amélioration de considérer cette variable qui a tellement de données manquantes et qui est difficile à calculer?



#-----------------------------------------------------------------------



#D.5 SVR (Méthode non linéaire):
    # "Support Vector Regression". Cette méthode cherche à minimiser les "pénalités" (L2 en particulier).
    #SVR nous donne la flexibilité de définir la quantité d'erreur acceptable dans notre modèle et trouvera une ligne 
    #appropriée (ou un hyperplan dans des dimensions plus élevées) pour s'adapter aux données.

In [191]:
SVR_E=SVR()

In [192]:
SVR_E.get_params()

{'C': 1.0,
 'cache_size': 200,
 'coef0': 0.0,
 'degree': 3,
 'epsilon': 0.1,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [ ]:
#Les hyperparamètres intéressant à être modifiés sont:

# -gamma:= Kernel coefficient. Il définit l'étendue de l'influence d'un seul exemple d'entraînement, les valeurs faibles 
#          signifiant "loin" et les valeurs élevées signifiant "proche". Les paramètres gamma peuvent être vus comme 
#          l'inverse du rayon d'influence des échantillons sélectionnés par le modèle comme vecteurs supports.

# -epsilon:= C'est le coefficient associé à la pénalité dans la fonction des pertes (loss fonction).

# -C:= Coefficient de régularisation. Il calibre le modèle et doit être positif pour éviter des over et under fit.
#      Le paramètre compromet la classification correcte des exemples d'apprentissage contre la maximisation de la marge de 
#      la fonction de décision. Pour des valeurs plus grandes de C, une marge plus petite sera acceptée si la fonction de 
#      décision réussit mieux à classer correctement tous les points d'apprentissage. Un C inférieur encouragera une plus 
#      grande marge, donc une fonction de décision plus simple, au détriment de la précision de l'apprentissage.



#D.5.0 Optimisation des hyperparamètres.
#On commence par déclencher le compteur de temps, pour mesurer le temps mis pour l'entraînement.

In [193]:
start_time_E = datetime.now()

In [194]:
parameters = {'gamma' : [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1],'epsilon' : [0.001, 0.01, 0.1, 1], 'C' : [0.001, 0.01, 0.1, 1, 10]}

SVR_g_E= GridSearchCV(estimator=SVR_E, param_grid = parameters, cv=5, verbose=False)

In [ ]:
#Notez que dans le GridSearchCV 'cv' c'est le nombre de plis (ie. d'itérations).

#D.5.1Entraînement du modèle. 

In [195]:
SVR_g_E.fit(train_dataset_En,train_cible_En)

GridSearchCV(cv=5, estimator=SVR(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10],
                         'epsilon': [0.001, 0.01, 0.1, 1],
                         'gamma': [1e-08, 1e-07, 1e-06, 1e-05, 0.0001, 0.001,
                                   0.01, 0.1]},
             verbose=False)

In [196]:
end_time_E = datetime.now()

In [ ]:
#On arrête le compteur de temps, pour calculer finalement le temps écoulé:

In [197]:
SVR_time_E = end_time_E - start_time_E

In [ ]:
#R^2 obtenu suite à l'entraînement (avant la prédiction):

In [198]:
SVR_g_E.score(train_dataset_En,train_cible_En)

0.5638657463359121

In [ ]:
#Les meilleurs hyperparamètres.

In [199]:
SVR_g_E.best_params_

{'C': 10, 'epsilon': 0.1, 'gamma': 0.1}

In [ ]:
#D.5.2 Développement du meilleur modèle:

In [200]:
SVR_best_E=SVR(C=10, epsilon=0.1, gamma=0.1)

In [201]:
SVR_best_E.fit(train_dataset_En,train_cible_En)

SVR(C=10, gamma=0.1)

In [ ]:
#D.5.3 Prédiction.

In [202]:
SVR_best_prediction_E=SVR_best_E.predict(test_dataset_En)
SVR_best_prediction_E

array([ 0.78850978, -0.50420583, -0.96455691,  0.52303043, -0.15589902,
        0.44508193,  0.16067093,  1.0948576 , -0.47899295, -0.49056379,
        1.73600532, -0.16433104,  0.24287439, -0.58535748, -1.15025687,
        0.7969651 , -0.83776876,  0.40598614, -0.85900227,  1.46515747,
       -0.42123578, -0.4670434 ,  1.12362907,  1.02110895,  1.10181479,
        0.10517315,  0.22849955,  0.26727035,  0.50670286, -0.12483565,
       -0.03600818,  0.55667155,  0.00245348,  0.36228691, -0.55193525,
        0.69094701,  0.48891631,  0.0127918 ,  0.72321912,  0.12684683,
        0.41129385,  0.74286685,  0.10238897,  0.39163612,  0.40724522,
        0.37992131,  0.83951188,  0.59996671, -0.13935667,  0.26686725,
        0.25644365,  0.52065439,  0.33205716, -1.25202094, -0.37876335,
        0.36071483, -0.92885375, -0.08183407,  0.22952201,  1.16869816,
        0.22754521,  0.75898033, -0.8956337 ,  0.15997734, -0.51338319,
       -0.04312452,  0.08262748, -1.4654483 ,  0.64470242,  0.32

In [ ]:
#D.5.4 Calcul de différentes métriques suite à la prédiction du modèle.

#Calcul du R^2:

In [203]:
R2_SVR_best_E=r2_score(test_cible_En,SVR_best_prediction_E)
R2_SVR_best_E

0.3982436738714338

In [ ]:
#Calcul du MSE:

In [204]:
MSE_SVR_best_E=mean_squared_error(test_cible_En,SVR_best_prediction_E)

In [ ]:
#Calcul du RMSE:

In [205]:
RMSE_SVR_best_E=math.sqrt(MSE_SVR_best_E)

In [ ]:
#Calcul du MAE:

In [206]:
 MAE_SVR_best_E=mean_absolute_error(test_cible_En,SVR_best_prediction_E)

In [ ]:
#D.5.4 Résultats.

In [207]:
results_SVR_E= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_SVR_E= results_SVR_E.append({'Modèle':'SVR','R^2':R2_SVR_best_E,'MSE':MSE_SVR_best_E,'RMSE':RMSE_SVR_best_E,'MAE':MAE_SVR_best_E ,'Temps de calcul (s)':SVR_time_E.seconds},ignore_index=True)
results_SVR_E

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,SVR,0.398244,0.659681,0.812207,0.47223,174


In [ ]:
# Notez que:

# -la valeur absolue du R^2 est loin de 1 et de celle du random forest, mais plus proche que celles du dummy regressor, la
#  régression linéaire et le elasticNet.
# -la valeur du RMSE est supérieure à 0.5 et à de celle du random forest, mais plus proche que le dummy regressor, la
#  régression linéaire et le elasticNet.
# -La valeur du MAE est moins bonne celle du random forest, mais meilleure que celles du modèle naïf de dummy regressor, de 
#la régression linéaire et du elasticNet (ie. plus petite).

#  Ainsi, les valeurs des métriques antérieures montrent la faiblesse de ce modèle, qui est quand même bien meilleur que le 
#  dummy regressor, la régression linéaire, le elasticNet et le random forest.

# Notez qu'en considérant le ENERGYSTARScore on améliore un peu les valeurs des métriques, mais vaut-il vraiment le coup pour
#cette amélioration de considérer cette variable qui a tellement de données manquantes et qui est difficile à calculer?

#---------------------------------------------------------------------------------------------------------------------------


#D.6 Tableau résumé:


In [210]:
results_total_E= pd.DataFrame(columns=['Modèle','R^2','MSE','RMSE','MAE','Temps de calcul (s)'])
results_total_E= results_total_E.append({'Modèle':'Dummy Regressor','R^2':R2_DM_E,'MSE':MSE_DM_E,'RMSE':RMSE_DM_E,'MAE':MAE_DM_E ,'Temps de calcul (s)':DM_time_E.seconds},ignore_index=True)
results_total_E= results_total_E.append({'Modèle':'Linear Regression','R^2':R2_LR_E,'MSE':MSE_LR_E,'RMSE':RMSE_LR_E,'MAE':MAE_LR_E ,'Temps de calcul (s)':LR_time_E.seconds},ignore_index=True)
results_total_E= results_total_E.append({'Modèle':'ElasticNet','R^2':R2_EN_best_E,'MSE':MSE_EN_best_E,'RMSE':RMSE_EN_best_E,'MAE':MAE_EN_best_E ,'Temps de calcul (s)':EN_time_E.seconds},ignore_index=True)
results_total_E= results_total_E.append({'Modèle':'Random Forest','R^2':R2_RF_best_E,'MSE':MSE_RF_best_E,'RMSE':RMSE_RF_best_E,'MAE':MAE_RF_best_E,'Temps de calcul (s)':RF_time_E.seconds},ignore_index=True)
results_total_E= results_total_E.append({'Modèle':'SVR','R^2':R2_SVR_best_E,'MSE':MSE_SVR_best_E,'RMSE':RMSE_SVR_best_E,'MAE':MAE_SVR_best_E,'Temps de calcul (s)':SVR_time_E.seconds},ignore_index=True)
results_total_E

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Dummy Regressor,-0.064185,1.166623,1.080103,0.777485,0
1,Linear Regression,0.329891,0.734613,0.857096,0.559039,0
2,ElasticNet,0.329416,0.735134,0.857399,0.559058,15
3,Random Forest,0.581775,0.458483,0.677114,0.362651,266
4,SVR,0.398244,0.659681,0.812207,0.472230,174


In [ ]:
#Montrer les deux tableaux ensembles:

In [211]:
from IPython.display import display_html
from itertools import chain,cycle
def display_side_by_side(*args,titles=cycle([''])):
    html_str=''
    for df,title in zip(args, chain(titles,cycle(['</br>'])) ):
        html_str+='<th style="text-align:center"><td style="vertical-align:top">'
        html_str+=f'<h2>{title}</h2>'
        html_str+=df.to_html().replace('table','table style="display:inline"')
        html_str+='</td></th>'
    display_html(html_str,raw=True)

In [212]:
display_side_by_side(results_total,results_total_E, titles=['Sans ENERGYSTARScore','Avec ENERGYSTARScore'])

,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Dummy Regressor,-0.004197,1.035714,1.017700,0.735766,1
1,Linear Regression,0.144916,0.881921,0.939107,0.670013,1
2,ElasticNet,0.143766,0.883107,0.939738,0.668716,15
3,Random Forest,0.528152,0.486657,0.697608,0.400458,316
4,SVR,0.133011,0.894200,0.945621,0.638368,342
,Modèle,R^2,MSE,RMSE,MAE,Temps de calcul (s)
0,Dummy Regressor,-0.064185,1.166623,1.080103,0.777485,0
1,Linear Regression,0.329891,0.734613,0.857096,0.559039,0
2,ElasticNet,0.329416,0.735134,0.857399,0.559058,15
3,Random Forest,0.581775,0.458483,0.677114,0.362651,266


In [ ]:
#Ainsi, voyant les inconvénients niveau calcul et stabilité du 'ENERGYSTARScore' on préfère ne pas le considérer dans notre
#modèle car il représente plus d’inconvénients de calcul que d'apports bénéfiques dans nos modèle.
#Et bien sûr on reste avec le Random Forest.